# Figure 3

In [1]:
%run 'stability_fig_init.py'

In [6]:
n_features = 8
important_features = []
fig = make_subplots(
    rows=len(areas),
    row_titles=["<b> {} </b>".format(i) for i in area_names],
    vertical_spacing=0.2,
)

for j, area in enumerate(areas):
    shap_vals_abs = pd.DataFrame(index=input_cols, columns=targets)
    important_features = []
    for targ in targets:
        X_test = pd.read_hdf(version_folder.format(area, version) + "X_test_full.h5")

        with open(
            fit_folder.format(area, version, targ) + "shap_values_gtb_full.pkl", "rb"
        ) as handle:
            sh = pickle.load(handle)
        shap_vals = pd.DataFrame(
            data=sh.values, index=X_test.index, columns=sh.feature_names
        )
        shap_vals_abs.loc[:, targ] = shap_vals.abs().mean()

        important_features += list(
            shap_vals.abs().mean().sort_values(ascending=False).iloc[:n_features].index
        )

    important_features = np.unique(important_features)
    shap_vals_abs = shap_vals_abs / shap_vals_abs.sum()

    for k, targ in enumerate(targets):
        if j == 0:
            fig.add_trace(
                go.Bar(
                    x=[
                        input_col_names[i]
                        for i in shap_vals_abs.loc[important_features, targ].index
                    ],
                    y=shap_vals_abs.loc[important_features, targ],
                    name=target_name[targ],
                    marker_color=px.colors.qualitative.T10[k],
                    showlegend=True,
                    text=shap_vals_abs.loc[important_features, targ]
                    .rank(ascending=False)
                    .astype(int)
                    .apply(lambda x: x if x < 6 else ""),
                    textposition="outside",
                ),
                row=j + 1,
                col=1,
            )
        else:
            fig.add_trace(
                go.Bar(
                    x=[
                        input_col_names[i]
                        for i in shap_vals_abs.loc[important_features, targ].index
                    ],
                    y=shap_vals_abs.loc[important_features, targ],
                    name=target_name[targ],
                    marker_color=px.colors.qualitative.T10[k],
                    showlegend=False,
                    text=shap_vals_abs.loc[important_features, targ]
                    .rank(ascending=False)
                    .astype(int)
                    .apply(lambda x: x if x < 6 else ""),
                    textposition="outside",
                ),
                row=j + 1,
                col=1,
            )
    fig.update_layout(
        {
            "yaxis{}".format(j + 1): {
                "linewidth": 1,
                "linecolor": "black",
                "mirror": True,
                "ticks": "outside",
                "dtick": 0.1,
                "showline": True,
                "range": [[0, 0.29], [0, 0.42], [0, 0.27]][j],
            },
            "xaxis{}".format(j + 1): {
                "tickangle": 35,
                "linewidth": 1,
                "linecolor": "black",
                "mirror": True,
                "ticks": "outside",
                "showline": True,
            },
        }
    )

fig.update_layout(
    yaxis2_title="Feature Importance",
    yaxis2_title_standoff=40,
    font=dict(size=35, color="black"),
)
fig.update_annotations(font_size=35)
fig.update_layout(
    template="plotly_white", width=1240, height=1300, legend_orientation="h"
)

fig.update_layout(font=dict(size=19, color="black"), legend_orientation="h")
# fig.update_annotations(font_size=37)
# fig.update_layout(uniformtext_minsize=37)
# fig.update_layout(margin=dict(b=85))
# fig.update_layout(font=dict(size=20, color='black'))
fig.update_annotations(font_size=19)
fig.update_layout(legend_font_size=20)
# fig.update_layout(legend_y=-0.3)
fig.update_layout(margin_t=30)
fig.update_layout(margin_r=0)

fig.update_layout(legend=dict(yanchor="top", y=-0.2, xanchor="center", x=0.5))
# fig.show()


print("creating plot...")
scope = pio.kaleido.scope
scope._shutdown_kaleido()
pio.write_image(fig, "fig3.pdf", format="pdf", validate=False, engine="kaleido")
time.sleep(1)
pio.write_image(fig, "fig3.pdf", format="pdf", validate=False, engine="kaleido")
scope._shutdown_kaleido()


creating plot...
